In [1]:
import time
import json
import re

begin_time_start = time.time()

import yopmail
import reporting

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## Generate analysis report

In [2]:
document_report = {
    'generate_email': None,
    'process_index': None,
    'process_toolbox': None,
    'switch_tag': None,
    'process_test': None,
    'submit_test': None,
    'submit_mail': None,
    'content_mail': None,
    'result_test': None }

## Generate Yopmail email

In [3]:
email = yopmail.create_random_email()

document_report['generate_email'] = email

## Log in to Hightest

In [4]:
time_start = time.time()
try:
    options = Options()
    options.add_argument(f"--window-size={1400},{800}")

    driver = webdriver.Chrome(options=options)
    driver.get("https://hightest.nc")

    # Wait and click on Toolbox.
    toolbox_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[@title="Toolbox"]')))
    toolbox_link.click()

    document_report['process_index'] = reporting.success_message(time_start)
except:
    document_report['process_index'] = reporting.error_message(time_start, 'impossible to connect to hightest.nc.')

## CHANGE PAGE => https://hightest.nc/toolbox/ ##

time_start = time.time()
try:
    # Move to this element.
    ISTQB_TITLE_PATH = '//*[@id="page"]/div[2]/div/section/article/div[2]/div/div[1]/div'
    button_french = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ISTQB_TITLE_PATH)))
    driver.execute_script("arguments[0].scrollIntoView();", button_french)

    # Wait for the element to be clickable.
    FRENCH_BUTTON_PATH = '//*[@id="page"]/div[2]/div/section/article/div[2]/div/div[1]/div/div[2]/div/a[1]'
    toolbox_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, FRENCH_BUTTON_PATH)))
    toolbox_link.click()

    document_report['process_toolbox'] = reporting.success_message(time_start)
except:
    document_report['process_toolbox'] = reporting.error_message(time_start, 'impossible to access the test, check XPath of "Français" button.')


## CHANGE PAGE => https://hightest.nc/ressources/test-istqb.php ##


time_start = time.time()
try:
    if driver.current_url != 'https://hightest.nc/ressources/test-istqb.php':
        onglets = driver.window_handles
        driver.switch_to.window(onglets[1])

    # Wait submit appear.
    submit_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="submit"]')))

    document_report['switch_tag'] = reporting.success_message(time_start)
except:
    document_report['switch_tag'] = reporting.error_message(time_start, 'impossible to access to correct tab, check window_handles process.')

time_start = time.time()
try:
    dict_answers = {
        1: 1, 2: 2, 3: 1, 4: 2, 5: 2,
        6: 3, 7: 2, 8: 4, 9: 1, 10: 3,
        11: 4, 12: 2, 13: 3, 14: 2, 15: 4,
        16: 3, 17: 3, 18: 1, 19: 2, 20: 2
    }

    for question in dict_answers:
        answer = dict_answers[question]

        # Paths.
        MULTI_answerS_PATH = f'//*[@id="main_content"]/form/div[{question}]'
        RADIO_BUTTON_PATH = f'//*[@id="main_content"]/form/div[{question}]/input[{answer}]'

        # Move to multiple answers.
        question_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, MULTI_answerS_PATH)))
        driver.execute_script("arguments[0].scrollIntoView();", question_element)

        # Click the radio button.
        radio_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, RADIO_BUTTON_PATH)))
        radio_button.click()

    document_report['process_test'] = reporting.success_message(time_start)
except:
    document_report['process_test'] = reporting.error_message(time_start, 'impossible to complete the form, check XPaths of radios.')

time_start = time.time()
try:
    # Click the radio button.
    SUBMIT_PATH = f'//*[@id="submit"]' 
    radio_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, SUBMIT_PATH)))
    radio_button.click()

    document_report['submit_test'] = reporting.success_message(time_start)
except:
    document_report['submit_test'] = reporting.error_message(time_start, 'impossible to submit the form, check XPath of "Terminé !".')


## CHANGE PAGE => https://hightest.nc/ressources/reponses-test-istqb.php ##

time_start = time.time()
try:
    TEXT_FIELD_PATH = '//*[@id="email"]'
    SUBMIT_MAIL_PATH = '//*[@id="submitMail"]'

    text_field = driver.find_element(By.XPATH, TEXT_FIELD_PATH)
    text_field.send_keys(email['generated_email'])

    submit_mail = driver.find_element(By.XPATH, SUBMIT_MAIL_PATH)
    submit_mail.click()

    driver.quit()

    document_report['submit_mail'] = reporting.success_message(time_start)
except:
    document_report['submit_mail'] = reporting.error_message(time_start, 'impossible to submit the mail, check XPaths in reponses-test page.')

# Let the server send the mail. 
time.sleep(2)

## Read Yopmail email

In [5]:
report = yopmail.last_message_of(email['identifier_email'])

document_report['content_mail'] = report

try:
    # Final result test.
    good_answer = int(re.search(r'répondu à (\d+) question\(s\)', report['content']).group(1))
    percentage_success = int(re.search(r'soit (\d+) % de réussite.', report['content']).group(1))

    if percentage_success == 100:
        status_test = 'success'
    else:
        status_test = 'some bad answer(s)'

    document_report['result_test'] = {'status': 'success',
                                      'status_test': status_test,
                                      'good_answer': good_answer,
                                      'percentage_success': percentage_success,
                                      'full_execution_time': reporting.execution_time(begin_time_start)}
except:
    document_report['result_test'] = reporting.error_message(time_start, 'impossible to extract results, check regex.')

## Save the JSON data to a file

In [6]:
json_data = json.dumps(document_report, indent=4, ensure_ascii=False)

with open('report_quiz_ISTQB_FR.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)